# Analyis of grid hyperparameter search

In [ ]:
import json
import pathlib
import pandas as pd
import vaep.pandas

pd.options.display.max_columns = 30

In [ ]:
fname = "runs/experiment03/all_metrics.json"

with open(pathlib.Path(fname)) as f:
    metrics_dict = json.load(f)

In [ ]:
metrics = {}
for k, run_metrics in metrics_dict.items():
    metrics[k] = vaep.pandas.flatten_dict_of_dicts(run_metrics)

metrics = pd.DataFrame.from_dict(metrics, orient='index')
metrics

- options see [2Dline plot](https://matplotlib.org/stable/api/_as_gen/matplotlib.lines.Line2D.html#matplotlib.lines.Line2D)

In [ ]:
metrics["valid_pred_collab"]['intensity_predicted'].sort_values(
    'MSE').iloc[:10].plot(rot=90, 
                          x_compat=True, 
                          xticks=list(range(10)),
                          marker='o',
                          linestyle=''
                          
                         )